## Import Competition Data

In [1]:
import numpy as np
import pandas as pd

property_df  = pd.read_csv("data/properties_2017.csv",low_memory = False)
train_df = pd.read_csv("data/train_2017.csv", parse_dates=["transactiondate"])

In [2]:
property_df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [3]:
train_df.head()

,parcelid,logerror,transactiondate
0,14297519,0.025595,2017-01-01
1,17052889,0.055619,2017-01-01
2,14186244,0.005383,2017-01-01
3,12177905,-0.103410,2017-01-01
4,10887214,0.006940,2017-01-01


## Data Cleaning

In [4]:
# find columns with missing values
missing_percent = property_df.isnull().sum() * 100 / len(property_df)
missing_values_df = pd.DataFrame({'column_name': property_df.columns, 'percent_missing': missing_percent})
print(missing_values_df[missing_values_df.percent_missing > 0].sort_values('percent_missing', ascending=False))

                                               column_name  percent_missing
storytypeid                                    storytypeid        99.945632
basementsqft                                  basementsqft        99.945498
yardbuildingsqft26                      yardbuildingsqft26        99.911363
fireplaceflag                                fireplaceflag        99.827048
architecturalstyletypeid          architecturalstyletypeid        99.796966
typeconstructiontypeid              typeconstructiontypeid        99.774020
finishedsquarefeet13                  finishedsquarefeet13        99.743034
buildingclasstypeid                    buildingclasstypeid        99.573532
pooltypeid10                                  pooltypeid10        99.430326
decktypeid                                      decktypeid        99.417831
finishedsquarefeet6                    finishedsquarefeet6        99.280387
poolsizesum                                    poolsizesum        99.063452
pooltypeid2 

In [5]:
# remove features with more than 70% null values
threshold = 80
null_cols = missing_values_df[missing_values_df.percent_missing > threshold].column_name.tolist()

In [6]:
property_df = property_df.drop(null_cols, axis=1)

In [7]:
property_df.shape

(2985217, 34)

In [8]:
# impute values for geographic categorical features - use mode of broader geographic feature by grouping
# using mode across the feature itself would provide geographical values that don't make sense 
# eg: find most common zip code in the county to impute missing zip

def impute_geographical_feature(df, target_feature, group_feature):
    # mode of target feature grouped by the group geographic feature
    mode_per_group = df.groupby(group_feature)[target_feature].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    
    # impute missing values 
    missing_mask = df[target_feature].isnull() & df[group_feature].notnull()
    df.loc[missing_mask, target_feature] = df.loc[missing_mask, group_feature].map(mode_per_group)
    
    # drop rows where group feature is missing or if target feature still missing after imputation
    df.dropna(subset=[target_feature, group_feature], inplace=True)

geographical_features = ['fips', 'regionidneighborhood', 'censustractandblock', 'rawcensustractandblock',\
                         'regionidzip', 'regionidcity', 'regionidcounty']
for feature in geographical_features:
    group_feature = 'regionidcounty' if feature != 'regionidcounty' or feature != 'fips' else 'regionidcity'
    impute_geographical_feature(property_df, feature, group_feature)

In [9]:
property_df.shape

(2982285, 34)

In [10]:
# impute categorical feature values
from sklearn.impute import SimpleImputer

cat_imputer = SimpleImputer(strategy='most_frequent') # use mode here as well

categorical_features = ['airconditioningtypeid', 'buildingqualitytypeid', 'heatingorsystemtypeid',
                        'propertycountylandusecode', 'propertylandusetypeid', 'propertyzoningdesc',
                        'regionidcity', 'regionidcounty', 'regionidzip', 'unitcnt',
                        'censustractandblock', 'rawcensustractandblock']

# impute missing values for each feature
for col in categorical_features:
    if col in property_df.columns:
        property_df[[col]] = cat_imputer.fit_transform(property_df[[col]])

In [11]:
# now do the same for the remaining numerical features

num_imputer = SimpleImputer(strategy='median') 

numerical_features = ['basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 
                      'calculatedbathnbr', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 
                      'finishedsquarefeet12', 'finishedsquarefeet50', 'garagecarcnt', 'garagetotalsqft', 
                      'latitude', 'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum', 'roomcnt', 
                      'threequarterbathnbr', 'unitcnt', 'yearbuilt', 'numberofstories',
                      'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 
                      'landtaxvaluedollarcnt', 'taxamount', 'fullbathcnt']

for col in numerical_features:
    if col in property_df.columns:
        property_df[[col]] = num_imputer.fit_transform(property_df[[col]])

In [12]:
# confirm all missing values filled
print(property_df.isnull().sum())

parcelid                        0
airconditioningtypeid           0
bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedbathnbr               0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fullbathcnt                     0
garagecarcnt                    0
garagetotalsqft                 0
heatingorsystemtypeid           0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
propertylandusetypeid           0
propertyzoningdesc              0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidneighborhood            0
regionidzip                     0
roomcnt                         0
unitcnt                         0
yearbuilt                       0
numberofstories                 0
structuretaxvaluedollarcnt      0
taxvaluedollar

### Combining Training Table

In [13]:
# pulling property data into training data
combined_df = train_df.merge(property_df, how='left', on='parcelid')
combined_df.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,14297519,0.025595,2017-01-01,1.0,3.5,4.0,6.0,3.5,3100.0,3100.0,...,0.0,1.0,1998.0,1.0,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13
1,17052889,0.055619,2017-01-01,1.0,1.0,2.0,6.0,1.0,1465.0,1465.0,...,5.0,1.0,1967.0,1.0,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13
2,14186244,0.005383,2017-01-01,1.0,2.0,3.0,6.0,2.0,1243.0,1243.0,...,6.0,1.0,1962.0,1.0,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13
3,12177905,-0.103410,2017-01-01,1.0,3.0,4.0,8.0,3.0,2376.0,2376.0,...,0.0,1.0,1970.0,1.0,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13
4,10887214,0.006940,2017-01-01,1.0,3.0,3.0,8.0,3.0,1312.0,1312.0,...,0.0,1.0,1964.0,1.0,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13


In [14]:
# remove properties that were removed during data cleaning in the new combined DataFrame
final_property_features = [col for col in property_df.columns if col != 'parcelid']
# remove rows where all property features missing:
missing_property_data = combined_df[final_property_features].isnull().all(axis=1) 
combined_df = combined_df[~missing_property_data]

## Dimensionality Reduction

In [15]:
# convert datetime columns to numerical format (to enable scaling)
for col in combined_df.select_dtypes(include=['datetime']):
    combined_df[f'{col}_year'] = combined_df[col].dt.year
    combined_df[f'{col}_month'] = combined_df[col].dt.month
    combined_df[f'{col}_day'] = combined_df[col].dt.day
    combined_df[f'{col}_weekday'] = combined_df[col].dt.weekday
    combined_df.drop(columns=[col], inplace=True)

In [16]:
# convert categorical to numerical for scaling as well
# using label encoding since one-hot encoding creates 2k+ features, which makes dim. red. infeasible 
categorical_cols = combined_df.select_dtypes(include=['object']).columns

from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    combined_df[col] = le.fit_transform(combined_df[col])
    label_encoders[col] = le  # store to invert encoding later if needed
    
# NOTE: this can create an implied ordinal ordering - can change this if affecting linear model performance

In [17]:
# define levels: increasing number of components
# Baseline: 100% (no dimensionality reduction)
# Level 1: 10% 
# Level 2: 50%
# Level 3: 75%

features = combined_df.drop(columns=['logerror'])
target = combined_df['logerror']

components_10 = int(features.shape[1] * 0.1)
components_50 = int(features.shape[1] * 0.5)
components_75 = int(features.shape[1] * 0.75)

In [18]:
combined_df.columns

Index(['parcelid', 'logerror', 'airconditioningtypeid', 'bathroomcnt',
       'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fips',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'unitcnt', 'yearbuilt', 'numberofstories', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'censustractandblock', 'transactiondate_year',
       'transactiondate_month', 'transactiondate_day',
       'transactiondate_weekday'],
      dtype='object')

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [20]:
# key metrics: run time, and data size (memory of dataframe)
# functions to measure these
import time
def measure_runtime(func):
    start_time = time.time()
    result = func()
    end_time = time.time()
    runtime = end_time - start_time
    return result, runtime

def dataframe_memory(df):
    return df.memory_usage(deep=True).sum()

In [21]:
pca_results = {} # store new reduced datasets and metrics

In [22]:
def apply_pca(n_components):
    pca = PCA(n_components=n_components)
    pca_result, runtime = measure_runtime(lambda: pca.fit_transform(features_scaled))
    reduced_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(n_components)])
    reduced_df['logerror'] = target
    mem_usage = dataframe_memory(reduced_df)
    return reduced_df, runtime, mem_usage

In [23]:
# Baseline - no reduction
baseline_runtime_start = time.time()
baseline_mem_usage = dataframe_memory(combined_df)
baseline_runtime = time.time() - baseline_runtime_start
pca_results['Baseline'] = (combined_df, baseline_runtime, baseline_mem_usage)

In [24]:
# PCA for 10%, 50%, 75%
from sklearn.decomposition import PCA
for components, label in zip([components_10, components_50, components_75], ['10%', '50%', '75%']):
    reduced_df, runtime, mem_usage = apply_pca(components)
    pca_results[label] = (reduced_df, runtime, mem_usage)

In [25]:
# Output Results
for label, (df, runtime, mem_usage) in pca_results.items():
    print(f"Results for {label}:")
    print(f"Runtime: {runtime:.4f} seconds")
    print(f"Data Size: {mem_usage} bytes")
    print(f"DataFrame head:\n{df.head()}\n")

Results for Baseline:
Runtime: 0.0015 seconds
Data Size: 23584016 bytes
DataFrame head:
   parcelid  logerror  airconditioningtypeid  bathroomcnt  bedroomcnt  \
0  14297519  0.025595                    1.0          3.5         4.0   
1  17052889  0.055619                    1.0          1.0         2.0   
2  14186244  0.005383                    1.0          2.0         3.0   
3  12177905 -0.103410                    1.0          3.0         4.0   
4  10887214  0.006940                    1.0          3.0         3.0   

   buildingqualitytypeid  calculatedbathnbr  calculatedfinishedsquarefeet  \
0                    6.0                3.5                        3100.0   
1                    6.0                1.0                        1465.0   
2                    6.0                2.0                        1243.0   
3                    8.0                3.0                        2376.0   
4                    8.0                3.0                        1312.0   

   finishe

### UMAP

In [26]:
umap_results = {}

In [27]:
import umap

def find_ideal_n_neighbors(components_list, features_scaled):
    avg_silhouette_scores = {}
    for n_neighbors in range(5, 21, 5):
        silhouette_scores = []
        for components in components_list:
            umap_model = umap.UMAP(n_neighbors=n_neighbors, n_components=components)
            umap_result = umap_model.fit_transform(features_scaled)
            silhouette_score = silhouette_score_umap(umap_result)
            silhouette_scores.append(silhouette_score)
        avg_silhouette_scores[n_neighbors] = np.mean(silhouette_scores)
    ideal_n_neighbors = max(avg_silhouette_scores, key=avg_silhouette_scores.get)
    return ideal_n_neighbors

In [28]:
from sklearn.metrics import silhouette_samples
def silhouette_score_umap(embedding):
    silhouette_vals = silhouette_samples(embedding, target)
    return np.mean(silhouette_vals)

In [29]:
def apply_umap(n_components, n_neighbors):
    umap_model = umap.UMAP(n_components=n_components, n_neighbors=n_neighbors)
    umap_result, runtime = measure_runtime(lambda: umap_model.fit_transform(features_scaled))
    reduced_df = pd.DataFrame(umap_result, columns=[f'UMAP_{i+1}' for i in range(n_components)])
    reduced_df['logerror'] = target
    mem_usage = reduced_df.memory_usage(deep=True).sum()
    return reduced_df, runtime, mem_usage

In [30]:
# ideal_n_neighbors = find_ideal_n_neighbors([components_10, components_50, components_75], features_scaled)
# print(ideal_n_neighbors)
# Found that ideal_n_neighbors = 10
ideal_n_neighbors = 10

In [31]:
# UMAP for 10%, 50%, 75%
for components, label in zip([components_10, components_50, components_75], ['10%', '50%', '75%']):
    reduced_df, runtime, mem_usage = apply_umap(components, ideal_n_neighbors)
    umap_results[label] = (reduced_df, runtime, mem_usage)

/Users/vincent/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [32]:
# Output Results
for label, (df, runtime, mem_usage) in umap_results.items():
    print(f"Results for {label}:")
    print(f"Runtime: {runtime:.4f} seconds")
    print(f"Data Size: {mem_usage} bytes")
    print(f"DataFrame head:\n{df.head()}\n")

Results for 10%:
Runtime: 26.2610 seconds
Data Size: 1551712 bytes
DataFrame head:
      UMAP_1    UMAP_2    UMAP_3  logerror
0  14.126016  2.504582 -4.334868  0.025595
1   9.245716 -4.616856  7.699994  0.055619
2  14.095885  4.228088 -0.367439  0.005383
3  12.117805 -3.276806  5.934363 -0.103410
4   1.742499  0.687398  1.563877  0.006940

Results for 50%:
Runtime: 19.2061 seconds
Data Size: 6516768 bytes
DataFrame head:
      UMAP_1    UMAP_2     UMAP_3    UMAP_4    UMAP_5    UMAP_6    UMAP_7  \
0   6.732989  7.516021   5.799178  4.018641  6.408525  7.533390  5.452029   
1   7.653164  4.262834   5.803383  2.486136  5.231318  5.245515  0.716009   
2   7.402006  6.651812   5.644090  3.180737  5.753782  7.609628  5.253371   
3   9.653627  5.367663  14.229658  2.844797  4.971064  5.183249  5.107074   
4  13.441078  4.836848   5.019262  2.339529  5.080245  4.703660  5.146778   

     UMAP_8    UMAP_9   UMAP_10   UMAP_11   UMAP_12   UMAP_13   UMAP_14  \
0  5.231379  9.742784  3.675102  5.14

### Autoencoder

In [33]:
autoencoder_results = {}

In [34]:
import tensorflow as tf
def autoencoder_dimension_reduction(dataset, n_dimensions):
    input_dim = dataset.shape[1]
    encoding_dim = n_dimensions

    input_data = tf.keras.layers.Input(shape=(input_dim,))
    encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_data)
    decoded = tf.keras.layers.Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = tf.keras.models.Model(input_data, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    autoencoder.fit(dataset, dataset, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

    encoder = tf.keras.models.Model(input_data, encoded)
    reduced_dataset = encoder.predict(dataset)
    return reduced_dataset


In [35]:
def apply_autoencoder(n_components):
    autoencoder_result, runtime = measure_runtime(lambda: autoencoder_dimension_reduction(features_scaled, n_components))
    reduced_df = pd.DataFrame(autoencoder_result, columns=[f'AE_{i+1}' for i in range(n_components)])
    reduced_df['logerror'] = target
    mem_usage = reduced_df.memory_usage(deep=True).sum()
    return reduced_df, runtime, mem_usage

In [36]:
# Autoencoder for 10%, 50%, 75%
for components, label in zip([components_10, components_50, components_75], ['10%', '50%', '75%']):
    reduced_df, runtime, mem_usage = apply_autoencoder(components)
    autoencoder_results[label] = (reduced_df, runtime, mem_usage)

Epoch 1/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - loss: 0.5719 - val_loss: 0.1178
Epoch 2/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step - loss: -0.3841 - val_loss: -0.8552
Epoch 3/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step - loss: -1.8697 - val_loss: -2.2664
Epoch 4/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 457us/step - loss: -4.0780 - val_loss: -4.2738
Epoch 5/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - loss: -7.2560 - val_loss: -6.8544
Epoch 6/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - loss: -11.2579 - val_loss: -9.9663
Epoch 7/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - loss: -15.9662 - val_loss: -13.5772
Epoch 8/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - loss: -21.4217 - val_loss: -17.6479
Epoch 9/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - loss: -27.8063 - val_loss: -22.1219
Epoch 10/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step - loss: -34.5217 - val_loss: -27.0018
Epoch 11/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step - loss: -41.3769 - val_loss: -32.2747

243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - loss: -427.3693 - val_loss: -327.5660
Epoch 15/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 469us/step - loss: -489.2973 - val_loss: -370.6607
Epoch 16/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - loss: -548.7621 - val_loss: -415.7548
Epoch 17/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - loss: -615.5610 - val_loss: -462.8850
Epoch 18/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step - loss: -697.1949 - val_loss: -512.0328
Epoch 19/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - loss: -750.3596 - val_loss: -563.1483
Epoch 20/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step - loss: -835.3917 - val_loss: -616.1848
Epoch 21/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - loss: -907.3234 - val_loss: -671.1588
Epoch 22/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - loss: -990.7425 - val_loss: -728.2999
Epoch 23/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - loss: -1072.3247 - val_loss: -787.1666
Epoch 24/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step - loss

243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - loss: -2041.6246 - val_loss: -1475.9744
Epoch 27/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - loss: -2167.6021 - val_loss: -1575.8164
Epoch 28/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - loss: -2284.0408 - val_loss: -1678.8539
Epoch 29/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - loss: -2493.3818 - val_loss: -1784.4718
Epoch 30/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - loss: -2598.7036 - val_loss: -1892.9010
Epoch 31/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - loss: -2742.7622 - val_loss: -2004.2393
Epoch 32/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - loss: -2927.8337 - val_loss: -2118.2930
Epoch 33/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - loss: -3149.7512 - val_loss: -2234.9307
Epoch 34/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - loss: -3283.4846 - val_loss: -2354.2441
Epoch 35/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - loss: -3464.6821 - val_loss: -2476.4050
Epoch 36/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 0

In [37]:
# Output Results
for label, (df, runtime, mem_usage) in autoencoder_results.items():
    print(f"Results for {label}:")
    print(f"Runtime: {runtime:.4f} seconds")
    print(f"Data Size: {mem_usage} bytes")
    print(f"DataFrame head:\n{df.head()}\n")

Results for 10%:
Runtime: 7.3601 seconds
Data Size: 1551712 bytes
DataFrame head:
         AE_1        AE_2        AE_3  logerror
0    0.000000   78.649483    0.000000  0.025595
1    0.000000    0.000000  235.044647  0.055619
2    0.000000    0.000000  138.575562  0.005383
3  131.508377   77.142845    0.000000 -0.103410
4  149.567780  140.925964    0.000000  0.006940

Results for 50%:
Runtime: 7.5417 seconds
Data Size: 6516768 bytes
DataFrame head:
        AE_1       AE_2       AE_3        AE_4        AE_5        AE_6  \
0   0.000000  58.471478   0.000000    0.000000   78.115265  167.026398   
1   0.000000   0.000000  62.474895  370.141998    0.000000  247.885086   
2  32.829689   0.000000  98.217300  164.613831    0.000000   87.327583   
3  16.429502  32.687943  71.749313    0.000000  245.528229    0.000000   
4   0.000000  79.137222  31.110634    0.000000  142.149780    0.000000   

         AE_7        AE_8        AE_9       AE_10       AE_11      AE_12  \
0    0.000000    0.000000 